In [1]:

import cv2 as cv
import numpy as np
from math import sqrt
import math




def seg_color(src,bajos, altos):
    mask = cv.inRange(src, bajos, altos)
    return mask
        

def filtro_area(mask):
    
    moments = cv.moments(mask)
    area=moments['m00']
    contours,hierarchy = cv.findContours(mask, 1, 2)
    cnt = contours[0]
    M = cv.moments(cnt)
    area = cv.contourArea(cnt)
    print(area)

    rect = cv.minAreaRect(cnt)
    box = cv.boxPoints(rect)
    box = np.int0(box)

#print( M )
    
    
#print(moments)      
    if(area > 2000):
#centr
        x = int(moments['m10']/moments['m00'])
        y = int(moments['m01']/moments['m00'])
#mostrar
        #print("x =", x)
        #print("y =", y)

        cv.rectangle(mask,(x,y),(x+2, y+2),(0,0,0),2)
        cv.drawContours(mask,[box],0,(255,255,255),2)
        x1,y1,w1,h1 = cv.boundingRect(cnt)
        cv.rectangle(mask,(x1,y1),(x1+w1,y1+h1),(255,255,255),2)
    
        return mask,x,y 
    
    
    
def obstaculo(maskAzul):
    kernel = np.ones((5,5),np.uint8)

    dilation = cv.dilate(maskAzul,kernel,iterations = 5)
#propiedades del contorno del obstaculo 
    contoursd,hierarchyd = cv.findContours(dilation, 1, 2)
    cntd = contoursd[0]
    leftmost = tuple(cntd[cntd[:,:,0].argmin()][0])
    rightmost = tuple(cntd[cntd[:,:,0].argmax()][0])
    topmost = tuple(cntd[cntd[:,:,1].argmin()][0])
    bottommost = tuple(cntd[cntd[:,:,1].argmax()][0])
##dibujando puntos
    cv.rectangle(dilation,leftmost,(leftmost[0]+2, leftmost[1]+2),(0,0,0),2)
    cv.rectangle(dilation,rightmost,(rightmost[0]+2, rightmost[1]+2),(0,0,0),2)
    cv.rectangle(dilation,topmost,(topmost[0]+2, topmost[1]+2),(0,0,0),2)
    cv.rectangle(dilation,bottommost,(bottommost[0]+2, bottommost[1]+2),(0,0,0),2)
    cv.rectangle(dilation,bottommost,(bottommost[0]+2, bottommost[1]+2),(0,0,0),2)
    #cv.rectangle(maskAmarillo,(x,y),(x+2, y+2),(0,0,0),2)
    
    return dilation,leftmost,rightmost,topmost,bottommost

      
def trayectoria(dilation,x,y,leftmost,rightmost,topmost,bottommost):
    trayect=[[x,y],[leftmost[0],topmost[1]],[rightmost[0],topmost[1]],[dilation.shape[0]-x,dilation.shape[1]-y]]
    return trayect

rojo_bajos = np.array([0,0,150], dtype=np.uint8)
rojo_altos = np.array([255, 122, 255], dtype=np.uint8)

verde_bajos = np.array([0,200,0], dtype=np.uint8)
verde_altos = np.array([255, 255, 100], dtype=np.uint8)

azul_bajos = np.array([100,0,0], dtype=np.uint8)
azul_altos = np.array([255, 80, 50], dtype=np.uint8)

amarillo_bajos = np.array([0,200,150], dtype=np.uint8)
amarillo_altos = np.array([100, 255, 255], dtype=np.uint8)

src=cv.imread('image.jpg')

maskRojo=seg_color(src, rojo_bajos, rojo_altos)    

maskVerde=seg_color(src, verde_bajos, verde_altos)

maskAzul=seg_color(src, azul_bajos, azul_altos)

maskAmarillo=seg_color(src, amarillo_bajos, amarillo_altos)
###

maskRojo,xRojo,YRojo=filtro_area(maskRojo)    

maskVerde,xVerde,yVerde=filtro_area(maskVerde)    
###
maskAzul1,xAzul,yAzul=filtro_area(maskAzul)    

maskAmarillo,xAmarillo,yAmarillo=filtro_area(maskAmarillo)  


###  
dilation,leftmost,rightmost,topmost,bottommost= obstaculo(maskAzul)
###
tRojo=trayectoria(dilation, xRojo, YRojo, leftmost, rightmost, topmost, bottommost)
tVerde=trayectoria(dilation, xVerde, yVerde, leftmost, rightmost, topmost, bottommost)
tAzul=trayectoria(dilation, xAzul, yAzul, leftmost, rightmost, topmost, bottommost)
tAmarillo=trayectoria(dilation, xAmarillo, yAmarillo, leftmost, rightmost, topmost, bottommost)
###
cv.imshow('original',src)
cv.imshow('amarillo', maskAmarillo)
cv.imshow('dilation', dilation)
print(tRojo)
cv.waitKey(0)
cv.destroyAllWindows()

3904.0
2907.5
15399.0
2079.5
[[119, 141], [281, 151], [363, 151], [481, 659]]
